# Hyperparameter Grid Search

In [ ]:
import jax
import jax.numpy as jnp
import pandas as pd
from scipy.stats import qmc
from functools import partial
import optax
import os
import sys

In [ ]:
optimizer = "adagrad"

# Variable bounds

$c\in (1, 4)$

$AR\in (5, 15)$

$\alpha_{L=0}\in (-6^{\circ}, 0^{\circ})$

$\alpha_{geo}\in (-10^{\circ}, 10^{\circ})$

In [ ]:
def deg2rad(rad):
    return jnp.deg2rad(rad).item()

In [ ]:
sampler = qmc.LatinHypercube(d=4)
sample = sampler.random(n=5000)

l_bounds = [1, 5, deg2rad(-6), deg2rad(-10)]
u_bounds = [4, 15, deg2rad(0), deg2rad(10)]

sample = qmc.scale(sample, l_bounds, u_bounds)
sample = jnp.array(sample)

#Turn the second column into the wingspan
c = sample[:, 0].reshape(-1, 1)
b = (sample[:, 0] * sample[:, 1]).reshape(-1, 1)
rest = sample[:, 2:]

sample = jnp.hstack([c, b, rest])

# Define the circulation solvers
A custom function is used to allow flexibility in using different hyperparameters for the lifting-line solver

In [ ]:
#Calculate induced angle of attack from Fourier coefficients
@jax.jit
def alpha_i_fn(theta, coefficients, n_list):
    summation_fn = jax.vmap(lambda An, n: n * An * jnp.sin(n * theta) / jnp.sin(theta))
    radians = summation_fn(coefficients, n_list).sum()   
    return radians

In [ ]:
@jax.jit
def circulation_error(theta, coefficients, n_list, alpha_0, alpha_geo, b, c):
    alpha_eff = alpha_0

    summation_fn = jax.vmap(lambda An, n: An * jnp.sin(n * theta))

    alpha_eff += (2*b)/(jnp.pi * c) * summation_fn(coefficients, n_list).sum()

    #summation_fn = jax.vmap(lambda An, n: n * An * jnp.sin(n * theta) / jnp.sin(theta))
    alpha_i = alpha_i_fn(theta, coefficients, n_list)

    error = jnp.rad2deg(alpha_eff + alpha_i - alpha_geo)

    return error

def circulation_loss(coefficients, n_list, wing_points, alpha_0, alpha_geo, b, c):
    thetas = jnp.linspace(1e-6, jnp.pi - 1e-6, wing_points)
    error = jax.vmap(circulation_error, in_axes=(0, None, None, None, None, None, None))

    loss = jnp.mean(error(thetas, coefficients, n_list, alpha_0, alpha_geo, b, c) ** 2) #MSE loss
    
    return loss

In [ ]:
def solve_coefficients(c, b, alpha_0, alpha_geo, num_coefficients, wing_points, lr, iters):
    num_coefficients = num_coefficients.astype(jnp.int32)
    wing_points = wing_points.astype(jnp.int32)
    iters = iters.astype(jnp.int32)
    
    n_list = jnp.arange(1, num_coefficients * 2 + 1, 2)
        
    #Initial guess for Fourier coefficients
    coefficients = jnp.zeros(num_coefficients)
    if optimizer == "adabelief":
        solver = optax.adabelief(lr)
    elif optimizer == "adam":
        solver = optax.adam(lr)
    elif optimizer == "adagrad":
        solver = optax.adagrad(lr)
    
    opt_state = solver.init(coefficients)
    
    value_and_grad = jax.value_and_grad(circulation_loss)
    cost = 1e10

    for i in range(iters):
        cost, grad = value_and_grad(coefficients, n_list, wing_points, alpha_0, alpha_geo, b, c)

        updates, opt_state = solver.update(
            grad,
            opt_state
        )

        coefficients = optax.apply_updates(coefficients, updates)
        
    #Check the loss for a constant wing points
    cost = circulation_loss(coefficients, n_list, 100, alpha_0, alpha_geo, b, c)
            
    return jnp.rad2deg(cost)

# Hyperparameter lists
TODO: Do these tests with different optimizer too (SGD, Adam, LBFGS, Gradient Descent, other Newton method)

In [ ]:
num_coefficients = jnp.array([5, 10, 20, 35, 50])
wing_points = jnp.array([10, 25, 50, 75, 100])
lr = jnp.array([1e-4, 5e-4, 1e-3, 5e-3, 1e-2])
iters = jnp.array([25, 50, 100, 150, 250])

hyperparams = jnp.meshgrid(num_coefficients, wing_points, lr, iters, indexing="ij")
hyperparams = jnp.stack(hyperparams, axis=-1)
hyperparams = hyperparams.reshape(-1, hyperparams.shape[-1])

hyperparams.shape

# Function to get loss of each combination

In [ ]:
def get_hyperparam_loss(params, sample):
    coefficients_losses = jax.vmap(solve_coefficients, in_axes=(0, 0, 0, 0, None, None, None, None))
    c = sample[:, 0]
    b = sample[:, 1]
    alpha_0 = sample[:, 2]
    alpha_geo = sample[:, 3]
    
    losses = coefficients_losses(c, b, alpha_0, alpha_geo, params[0], params[1], params[2], params[3])
    
    return jnp.mean(losses)

# Run the grid search

In [ ]:
import time
import pandas as pd

losses = []
times = []

for i in range(hyperparams.shape[0]):
    start = time.time()
    loss = get_hyperparam_loss(hyperparams[i], sample)
    end = time.time()
    
    times.append(end-start)
    losses.append(loss)
    
    print(i)
    
losses = jnp.array(losses).reshape(-1, 1)
times = jnp.array(times).reshape(-1, 1)

total = jnp.hstack((hyperparams, losses, times))

df = pd.DataFrame(total, columns=["num_coefficients", "wing_points", "lr", "iters", "loss", "time"])
df.to_csv(f"/kaggle/working/gridsearch_{optimizer}.csv")